In [372]:
import numpy as np
import pandas as pd

In [397]:
df = pd.read_csv('data/df_with_neis_details.csv')
df
df['start_dt'] = pd.to_datetime(df['start_dt'], errors='coerce')
df['end_dt'] = pd.to_datetime(df['end_dt'],errors='coerce')

In [439]:
def hour_range_list(end_dt,next_start_dt,start_dt,dest_nei_id, origin_nei_id):
    l = pd.period_range(end_dt, next_start_dt, freq='H').hour.values.tolist()
    if origin_nei_id == dest_nei_id and start_dt.hour in l:
        l.remove(start_dt.hour)
    if next_start_dt.hour in l:
        l.remove(next_start_dt.hour)
    return l

In [436]:
def diff_stay(dest_nei_id, origin_nei_id, end_dt, next_start_dt, start_dt):
    l = [end_dt.hour]
    if origin_nei_id == dest_nei_id and end_dt.hour == start_dt.hour:
        return []
    else:
        return [end_dt.hour]
    
    if origin_nei_id == dest_nei_id and end_dt.hour != start_dt.hour:
        return [end_dt.hour]
    elif origin_nei_id != dest_nei_id and end_dt.hour != start_dt.hour:
        return [end_dt.hour]
    elif origin_nei_id != dest_nei_id and end_dt.hour == start_dt.hour:
        return [end_dt.hour]

In [440]:
gb = df.groupby('vehicle_id')
groups_vehicle_id = [gb.get_group(x).sort_values(by=['start_dt']) for x in gb.groups]
groups_vehicle_id_shift = [gb.get_group(x).shift(periods=1).sort_values(by=['start_dt']) for x in gb.groups]

In [441]:
for g in groups_vehicle_id:
    g_shift = g.shift(periods=-1).sort_values(by=['start_dt'])
    g["next_origin_nei_id"] = g_shift["origin_nei_id"]
    g["next_start_dt"] = g_shift["start_dt"]
    g['hour_list'] = g.apply(lambda x: hour_range_list(x.end_dt, x.next_start_dt, x.start_dt,x.dest_nei_id, x.origin_nei_id) if x.dest_nei_id == x.next_origin_nei_id else diff_stay(x.dest_nei_id, x.origin_nei_id, x.end_dt, x.next_start_dt, x.start_dt), axis=1)

In [449]:
k = groups_vehicle_id[191][["start_dt","origin_nei_id" ,"end_dt","dest_nei_id", "next_start_dt", "next_origin_nei_id","hour_list","start_hour"]]
k.head(20)


,start_dt,origin_nei_id,end_dt,dest_nei_id,next_start_dt,next_origin_nei_id,hour_list,start_hour
49666,2017-10-23 07:46:54,58.0,2017-10-23 08:47:42.353,58.0,2017-10-23 19:11:05,58.0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",7
49669,2017-10-23 19:11:05,58.0,2017-10-23 23:06:48.320,58.0,2017-10-24 09:18:09,58.0,"[23, 0, 1, 2, 3, 4, 5, 6, 7, 8]",19
96082,2017-10-24 09:18:09,58.0,2017-10-24 09:42:43.217,40.0,2017-10-24 15:42:15,19.0,[9],9
65119,2017-10-24 15:42:15,19.0,2017-10-24 16:11:53.107,30.0,2017-10-24 16:56:00,30.0,[],15
104914,2017-10-24 16:56:00,30.0,2017-10-24 17:18:24.760,19.0,2017-10-25 10:00:26,19.0,"[17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5,...",16
43166,2017-10-25 10:00:26,19.0,2017-10-25 10:23:29.287,10.0,2017-10-26 06:50:28,40.0,[10],10
103829,2017-10-26 06:50:28,40.0,2017-10-26 07:01:41.947,35.0,2017-10-26 07:16:33,35.0,[],6
45215,2017-10-26 07:16:33,35.0,2017-10-26 07:40:49.307,11.0,2017-10-26 07:43:25,11.0,[],7
44450,2017-10-26 07:43:25,11.0,2017-10-26 07:45:01.940,11.0,2017-10-26 11:15:49,11.0,"[8, 9, 10]",7
69060,2017-10-26 11:15:49,11.0,2017-10-26 11:31:14.693,34.0,2017-10-26 16:43:28,34.0,"[11, 12, 13, 14, 15]",11


In [450]:
r = pd.concat(groups_vehicle_id)
r = r.sort_index()

In [451]:
(r["order_id"]==df["order_id"]).sum() == df.shape[0]

True

In [452]:
r[r.hour_list.apply(lambda c: c == np.nan)]


,vehicle_id,order_id,shiryun_cat,avg_dist,billing_minutes,orig_addr,dest_addr,start_dt,end_dt,kmh,...,orig_nei_name,orig_nei_area,orig_nei_popu,dest_nei_id,dest_nei_name,dest_nei_area,dest_nei_popu,next_origin_nei_id,next_start_dt,hour_list


In [453]:
df.to_csv('data/df_with_hour_list.csv', index=False)